In [33]:
########## Calling relevant libraries ##########
%matplotlib inline
import numpy as np
from sklearn.metrics import mean_squared_error
import scipy

# Setting matplotlib options for this notebook
import matplotlib
from matplotlib import pyplot as plt

import sklearn
from sklearn.externals import joblib

#import neuravi

from __future__ import absolute_import, division, print_function

import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()
tf.executing_eagerly() 

AttributeError: module 'pandas.core.computation' has no attribute 'expressions'

In [ ]:
size_input  = 1
size_hidden = 100

X_train = np.random.randn(800,1)
X_test = np.random.randn(200,1)
y_train = 2.0*X_train**2 + 0.2
y_test = 2.0*X_test**2 + 0.2

# Split test dataset into batches (training dataset will be randomly split into batches during each epoch of model training)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(4)
plt.scatter(X_train,y_train)


In [34]:
# Define class to build model
class Model(object):
    def __init__(self, size_input, size_hidden, size_output, device=None):
        """
        size_input: int, size of input layer
        size_hidden: int, size of hidden layer
        size_output: int, size of output layer
        device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
        """
        self.size_input, self.size_hidden, self.size_output, self.device =\
        size_input, size_hidden, size_output, device

        # Initialize weights between input layer and hidden layer
        self.W_xh = tfe.Variable(tf.random_normal([self.size_input, self.size_hidden]))
        # Initialize weights between hidden layer and output layer
        self.W_hy = tfe.Variable(tf.random_normal([self.size_hidden, self.size_output]))
        # Initialize biases for hidden layer
        self.b_h = tfe.Variable(tf.random_normal([1, self.size_hidden]))
        # Initialize biases for output layer
        self.b_y = tfe.Variable(tf.random_normal([1, self.size_output]))

        # Define variables to be updated during backpropagation
        self.variables = [self.W_xh, self.W_hy, self.b_h, self.b_y]

    def forward(self, X):
        """
        Method to do forward pass
        X: Tensor, inputs
        """
        if self.device is not None:
          with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
            self.y = self.compute_output(X)
        else:
          # Leave choice of device to default
          self.y = self.compute_output(X)

        return self.y

    def loss(self, y_pred, y_true):
        '''
        Method to do backpropagation of loss
        y_pred - Tensor of shape (batch_size, size_output)
        y_true - Tensor of shape (batch_size, size_output)
        '''
        y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
        # Cast y_pred to float32
        y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
        return tf.losses.mean_squared_error(y_true_tf, y_pred_tf)

    def backward(self, X_train, y_train):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-4)
        with tf.GradientTape() as tape:
          predicted = self.forward(X_train)
          current_loss = self.loss(predicted, y_train)
        grads = tape.gradient(current_loss, self.variables)
        optimizer.apply_gradients(zip(grads, self.variables),
                                  global_step=tf.train.get_or_create_global_step())
        #     print('Loss: {:.3f}'.format(self.loss(self.forward(X_train), y_train)))


    def compute_output(self, X):
        """
        Custom method to obtain output tensor during forward pass
        """
        # Cast X to float32
        X_tf = tf.cast(X, dtype=tf.float32)
        # Compute values in hidden layer
        a = tf.matmul(X_tf, self.W_xh) + self.b_h
        l_h = tf.nn.relu(a)
        # Compute output
        output = tf.matmul(l_h, self.W_hy) + self.b_y
        return output

In [35]:
# Set number of epochs
NUM_EPOCHS = 5

# Initialize model, letting device be selected by default during Eager Execution
model_default = Model(size_input, size_hidden, 1)

time_start = time.time()
for epoch in range(NUM_EPOCHS):
  loss_total = tfe.Variable(0, dtype=tf.float32)
  train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10, seed=epoch).batch(4)
  for inputs, outputs in train_ds:
    preds = model_default.forward(inputs)
    loss_total = loss_total + model_default.loss(preds, outputs)
    model_default.backward(inputs, outputs)
  print('Epoch {} - Average MSE: {:.4f}'.format(epoch + 1, loss_total.numpy() / X_train.shape[0]))
time_taken = time.time() - time_start

print('\nTotal time taken for training (secoonds): {:.2f}'.format(time_taken))

NameError: name 'tfe' is not defined

In [13]:
network(x)

InvalidArgumentError: cannot compute MatMul as input #0(zero-based) was expected to be a float tensor but is a double tensor [Op:MatMul]